### Model of institution in and agent-based model.

This is an agent-based model (ABM) of a Public Healthcare Institution using Python's Mesa library. This model will simulate how access to healthcare services affects societal health outcomes.

**Model Concept: Healthcare Access Simulation**

Components:
- Citizens (agents) with varying health statuses and income levels.
- Hospitals (agents) providing healthcare services.
- Government policies (model-level parameters) affecting accessibility.

Key dynamics:
- Citizens' health deteriorates over time.
- Wealthier citizens access healthcare more easily.
- Hospital capacity limits service availability.
- Government funding affects service quality.

In [7]:
import mesa
import random
import numpy as np

class Citizen(mesa.Agent):
    def __init__(self, unique_id, model, income_level):
        super().__init__(unique_id, model)
        self.health = random.uniform(0.7, 1.0)  # 0.0-1.0 scale
        self.income_level = income_level  # Low:0, Medium:1, High:2
        
    def step(self):
        # Health naturally deteriorates
        self.health -= random.uniform(0.01, 0.05)
        
        # Prevent health from going below 0
        self.health = max(0.0, self.health)
        
        # Seek healthcare if health is poor
        if self.health < 0.6 and random.random() < 0.7:
            self.seek_treatment()
    
    def seek_treatment(self):
        # Find nearest hospital
        nearest_hospital = min(
            self.model.hospitals, 
            key=lambda h: self.distance_to(h)
        )
        nearest_hospital.request_treatment(self)
    
    def distance_to(self, other_agent):
        """Calculate Euclidean distance to another agent"""
        return np.sqrt((self.pos[0]-other_agent.pos[0])**2 + 
                      (self.pos[1]-other_agent.pos[1])**2)

class Hospital(mesa.Agent):
    def __init__(self, unique_id, model, capacity):
        super().__init__(unique_id, model)
        self.capacity = capacity
        self.current_patients = 0
        self.success_rate = 0.8  # Base success rate
        
    def request_treatment(self, patient):
        if self.current_patients < self.capacity:
            self.current_patients += 1
            # Wealthier patients get better care
            effectiveness = min(1.0, self.success_rate + patient.income_level*0.1)
            patient.health = min(1.0, patient.health + random.uniform(0.2, 0.4)*effectiveness
            
    def step(self):
        # Reset capacity each step
        self.current_patients = 0

class HealthcareModel(mesa.Model):
    def __init__(self, population=100, hospital_count=3, gov_funding=0.5):
        self.num_agents = population
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.MultiGrid(20, 20, True)
        self.hospitals = []
        
        # Create citizens
        for i in range(population):
            income = random.choices([0,1,2], weights=[0.4,0.35,0.25])[0]
            citizen = Citizen(i, self, income)
            self.schedule.add(citizen)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(citizen, (x, y))
        
        # Create hospitals
        for i in range(hospital_count):
            hospital = Hospital(1000+i, self, capacity=15+int(gov_funding*20))
            hospital.success_rate = 0.6 + gov_funding*0.4
            self.hospitals.append(hospital)
            self.schedule.add(hospital)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(hospital, (x, y))
    
    def step(self):
        self.schedule.step()

# Run the simulation
model = HealthcareModel(population=200, gov_funding=0.7)

for i in range(100):
    model.step()

# Calculate statistics
citizens = [a for a in model.schedule.agents if isinstance(a, Citizen)]
avg_health = sum([a.health for a in citizens])/len(citizens)
health_inequality = np.std([a.health for a in citizens])

print(f"Average health: {avg_health:.3f}")
print(f"Health inequality: {health_inequality:.3f}")
print(f"Low-income health: {sum([a.health for a in citizens if a.income_level==0])/len([a for a in citizens if a.income_level==0]):.3f}")
print(f"High-income health: {sum([a.health for a in citizens if a.income_level==2])/len([a for a in citizens if a.income_level==2]):.3f}")

SyntaxError: invalid syntax (2077240261.py, line 49)

In [4]:
def portray(agent):
    if type(agent) is Citizen:
        color = ["red", "yellow", "green"][agent.income_level]
        return {"Shape": "circle", "Color": color, "r": 0.5}
    else:
        return {"Shape": "rect", "Color": "blue", "w": 1, "h": 1}

grid = mesa.visualization.CanvasGrid(portray, 20, 20, 500, 500)
server = mesa.visualization.ModularServer(HealthcareModel, [grid], "Healthcare Model")
server.port = 8521
server.launch()

NameError: name 'mesa' is not defined